# RAG로 AI 와인 소믈리에 함수화 하기

## 환경변수 로딩

In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

## 필요한 모듈 로딩

In [2]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

## 1. LLM을 통한 요리 정보 해석
- 이미지 -> 맛과 풍미 (image to text)
- 입력 : 요리 이미지(url)
- 출력 : 요리명, 요리에 대한 풍미 설명
- 함수로 정의한 다음, RunnableLambda 객체 사용하기

In [3]:
# 함수 정의 : 이미지 -> 요리명, 풍미 설명 출력
def describe_dish_flavor(input_data):

    prompt = ChatPromptTemplate([
        ("system", """
        You are a culinary expert who analyzes food images.
        When a user provides an image of a dish,
        identify the commonly recognized name of the dish, and
        clearly and concisely describe its flavor, focusing on the cooking method, texture, aroma, and balance of taste.
        If there is any uncertainty, base your analysis on the most likely dish, avoid definitive claims, and maintain a professional, expert tone.
        """),
        HumanMessagePromptTemplate.from_template([
            {"text": """아래의 이미지의 요리에 대한 요리명과 요리의 풍미를 설명해 주세요.
            출력형태 :
            요리명:
            요리의 풍미:
            """},
            {"image_url": "{image_url}"} # image_url는 정해줘 있음.        
        ])
    ])
    
    # llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.1,
        api_key=GOOGLE_API_KEY
    )
    output_parser = StrOutputParser()
    
    chain = prompt | llm | output_parser

    return chain

In [4]:
from langchain_core.runnables import RunnableLambda

# 함수를 전달인자로 넣기
r1 = RunnableLambda(describe_dish_flavor)

# RunnableLambda를 통한 함수 실행
input_data = {
    "image_url": "https://thumbnail.coupangcdn.com/thumbnails/remote/492x492ex/image/vendor_inventory/9d0d/fd3f0d77757f64b2eba0905dcdd85051932ec1ab5e6afc0c3246f403fabc.jpg"
}

res = r1.invoke(input_data)

In [5]:
print(res)

요리명: 스테이크 (Steak)

요리의 풍미:
이 스테이크는 겉면이 완벽하게 시어링되어 깊고 고소한 캐러멜화된 풍미와 바삭한 식감을 선사합니다. 내부는 이상적인 미디엄 레어로 조리되어 매우 부드럽고 촉촉하며, 육즙이 풍부하여 소고기 본연의 진한 풍미를 온전히 느낄 수 있습니다. 굵은 소금은 육향을 극대화하고, 흑후추는 은은한 매콤함과 향긋함을 더하며 전체적인 맛의 균형을 잡아줍니다. 또한, 로즈마리 한 조각은 신선하고 상쾌한 허브 향을 더해 스테이크의 풍미를 한층 더 고급스럽고 다채롭게 완성합니다.


## 2. 요리에 가장 잘 어울리는 wine top 5 검색
- pinecone 벡터 db 저장되어 있음
- index : wine-reviews, namespace : wine-reviews-ns

In [8]:
# 사용자 프롬프트 vector화, 유사도 높은 top=5 찾기
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore


# 요리에 대한 풍미(설명) 들어오면,
# 벡터 db에 인덱할 때 사용한 동일 임베딩 모델을 사용해서 임베딩 벡터 생성
embedding = OpenAIEmbeddings(
     model = OPENAI_EMBEDDING_MODEL
 )

# 벡터 db에서 유사도계산, top-5 검색
# 벡터 db 객체 생성
vector_db = PineconeVectorStore(
    embedding = embedding,
    index_name = PINECONE_INDEX_NAME ,
    namespace = PINECONE_NAMESPACE
)
# 벡터 db에서 질문과 가장 유사한, top-5 검색하기
query = res  # 질문
print("질문 : ", query)
print("-"*50)
vector_db.similarity_search(query, k=5)  # top-5 검색

질문 :  요리명: 스테이크 (Steak)

요리의 풍미:
이 스테이크는 겉면이 완벽하게 시어링되어 깊고 고소한 캐러멜화된 풍미와 바삭한 식감을 선사합니다. 내부는 이상적인 미디엄 레어로 조리되어 매우 부드럽고 촉촉하며, 육즙이 풍부하여 소고기 본연의 진한 풍미를 온전히 느낄 수 있습니다. 굵은 소금은 육향을 극대화하고, 흑후추는 은은한 매콤함과 향긋함을 더하며 전체적인 맛의 균형을 잡아줍니다. 또한, 로즈마리 한 조각은 신선하고 상쾌한 허브 향을 더해 스테이크의 풍미를 한층 더 고급스럽고 다채롭게 완성합니다.
--------------------------------------------------


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
# 